# NEP and NET calculations from $P_\text{sat}, G, T_c, \kappa$ lab test data

$\bf{Inputs}$

Parameters:
- channels.txt (containing default channel, and MF90, MF150, HF150, HF220 channels from Suzanne's data)
- opticalsparams_*.txt (I generated one for each of MF90, MF150, HF150, HF220 from Suzanne's data)
- flink = 1
- tau = 0.5

Data: (three different formats read separately in individual blocks)
- MF1_lab_3n.txt
- MF2_lab_3n.txt
- MF1_parameters_pton_lab_170711_sc.dat
- MF2_parameters_pton_lab_170711_sc.dat
- hf2_param.txt
- OutputValues\_all\_*.txt (noise forecasts from running NETbook with parameters for each channel)

$\bf{Outputs}$
- per data set, a .pdf figure per data set of NEP distribution (with NETbook forecast marked), and of NET distribution
- fcastlab_compare.txt (contains freq, Psat, G, Tc, kappa, nep, nepG, nepD, nepS, net_single, net values for each data set and forecasted by NETbook.)
- estimate_safety.txt (estimate of fsafety from lab Psats and NETbook ptots)
- fcastlab_synthesise.txt (more realistic noise estimate combining NEP_dicke and NEP_shot from NETbook forecast and NEP_G from lab tests)

$\bf{Notes}$

- Presently removing non valid data (nan, 'NC', inf) is not done in a generalised manner; method is specific to the input files. 
- NEP/NET calculations and figure generation are executed for the set of data most recently read, whereas the writing to file blocks execute for all results of processed data that exist in the array outputs[]

In [1]:
# import packages, establish directory, create object 'results' which is used for reporting

import numpy as np
import os
import matplotlib.pyplot as plt
import datetime

os.chdir('/Users/Shiye/Desktop/summer17/NETbook')

class results:

    def __init__(self, label, freq, Psat, G, Tc, kappa, nep, nepG, nepD, nepS, net_single, net, r, etasky, nTES):
        self.label = label
        self.freq = freq
        self.Psat = Psat
        self.G = G
        self.Tc = Tc
        self.kappa = kappa
        self.nep = nep
        self.nepG = nepG
        self.nepD = nepD
        self.nepS = nepS
        self.net_single = net_single
        self.net = net
        self.r = r
        self.etasky = etasky
        self.nTES = nTES
    
    def __hash__(self):
        return hash(label)
    
    def __eq__(self, other):
        return self.label == other.label
    
output = [] # to be populated with array of 'results' objects

In [4]:
###################################
# input format: MF1, MF2 from Maria
###################################

array = input('Maria\'s MFs: array 1 or 2? ')

fn = 'MF' + array + '_lab_3n.txt'
label = 'mf' + array + '_maria'
fo = open(fn, 'r')

lines = fo.readlines()[1:]
ntrials = [0,0]
freq = [90, 150]

col = [[],[]]
row = [[],[]]
G = [[],[]]
Psat = [[],[]]
sigmaPsat = [[],[]]
kappa = [[],[]]
Tc = [[],[]]
n = [[],[]]

for line in lines:
    split = line.split('\t')
    try: # reject 'frequency = NC' lines
        if int(split[9]) == freq[0]:
            ind = 0 # low freq
            ntrials[0] += 1
        elif int(split[9]) == freq[1]:
            ind = 1 # high freq
            ntrials[1] += 1
        else:
            raise Exception('frequency does not match one of two bands')
    except ValueError:
        #print('NC faulty line at col =', split[1], ', row =', split[0])
        continue
    col[ind].append(int(split[0]))
    row[ind].append(int(split[1]))
    G[ind].append(float(split[2]))
    Psat[ind].append(float(split[3]))
    sigmaPsat[ind].append(float(split[4]))
    kappa[ind].append(float(split[5]))
    Tc[ind].append(float(split[6]))
    n[ind].append(float(split[7]))

G = [[i * 1e-12 for i in j] for j in G] # convert from pW per K
Psat = [[i * 1e-12 for i in j] for j in Psat] # convert from pW
sigmaPsat = [[i * 1e-12 for i in j] for j in sigmaPsat] # convert from pW
Tc = [[i * 1e-3 for i in j] for j in Tc]  # convert from mK
kappa = [[i * 1e-12 for i in j] for j in kappa] # convert from pW per K^n

Maria's MFs: array 1 or 2? 2


In [8]:
###################################
# input format: MF1, MF2 from Steve
###################################

array = input('Steve\'s MFs: array 1 or 2? ')

fn = 'MF' + array + '_parameters_pton_lab_170711_sc.dat'
label = 'mf' + array + '_steve'
fo = open(fn, 'r')

lines = fo.readlines()[1:] # remove header line
ntrials = [0,0]

freq = [90, 150]

col = [[],[]]
row = [[],[]]
Psat = [[],[]]
Tc = [[],[]]
G = [[],[]]
kappa = [[],[]]
n = [[],[]]
opt_map = [[],[]]

for line in lines:
    split = line.split('\t')
    if split[8] == 'dark\n': # check that opt_map is dark
        if not np.isfinite(float(split[6])): # reject nan lines
            #print('NAN faulty line at col =', split[0], ', row =', split[1])
            continue
        try: # reject 'frequency = NC' lines
            if int(split[7]) == freq[0]:
                ind = 0 # low freq
                ntrials[0] += 1
            elif float(split[7]) == freq[1]:
                ind = 1 # high freq
                ntrials[1] += 1
            else:
                raise Exception('frequency does not match one of two bands')
        except ValueError:
            #print('NC faulty line at col =', split[0], ', row =', split[1])
            continue
        col[ind].append(int(split[0]))
        row[ind].append(int(split[1]))
        Psat[ind].append(float(split[2])) 
        Tc[ind].append(float(split[3]))
        G[ind].append(float(split[4]))
        kappa[ind].append(float(split[5]))
        n[ind].append(float(split[6]))

Psat = [[i * 1e-12 for i in j] for j in Psat] # convert from pW
kappa = [[i * 1e-3 for i in j] for j in kappa] # convert from pW per K^n THIS WAS GUESSED FROM ORDER OF MAGNITUDE
Tc = [[i * 1e-3 for i in j] for j in Tc]  # convert from mK
G = [[i * 1e-12 for i in j] for j in G] # convert from pW per K

# manual inspection showed that in all nan lines, n is nan, thus checking n = nan was sufficient for this case
# this should probably be generalised in the future

Steve's MFs: array 1 or 2? 2


In [10]:
##############################
# input format: HF from Steve
##############################

fo = open('hf2_param.txt', 'r')
label = 'hf_steve'
lines = fo.readlines()[1:] # remove header line
ntrials = [0, 0]

freq = [150, 220]

col = [[],[]]
row = [[],[]]
n = [[],[]]
Psat = [[],[]]
kappa = [[],[]]
Tc = [[],[]]
G = [[],[]]

for line in lines:
    faulty = False # faulty if contains nan, +inf, -inf
    split = line.split('\t')
    for entry in split:
        if not np.isfinite(float(entry)):
            faulty = True
            #print('faulty line at: row =', split[0], ', col =', split[1])
            break
    if faulty:
        continue
    if int(split[5]) == freq[0]:
        ind = 0 # low freq 
        ntrials[0] += 1
    elif int(split[5]) == freq[1]:
        ind = 1 # high freq
        ntrials[1] +=1
    else:
        raise Exception('frequency does not match one of two bands')
    col[ind].append(int(split[0]))
    row[ind].append(int(split[1]))
    n[ind].append(float(split[2]))
    Psat[ind].append(float(split[3]))
    kappa[ind].append(float(split[4]))
    Tc[ind].append(float(split[6]))
    G[ind].append(float(split[7]))

Psat = [[i * 1e-12 for i in j] for j in Psat] # convert from pW
kappa = [[i * 1e-3 for i in j] for j in kappa] # convert from pW per K^n THIS WAS GUESSED FROM ORDER OF MAGNITUDE
G = [[i * 1e-12 for i in j] for j in G] # convert from pW per K

# MANUAL PRUNING: unlikely high Psat at col = 21, row = 50, (index 923)
prune = True # to remove point
if prune:
    tofix = [col, row, n, Psat, kappa, Tc, G]
    for array in tofix:
        array[0].pop(416)
    ntrials[0] -= 1

In [11]:
# NEP and NET calculation

debug = True # also prints the conversion factors for debugging

h = 6.626e-34 # Planck's constant
k = 1.381e-23 # Boltzmann's constant
Tcmb = 2.725 # CMB temperature
flink = 1 # default to 1 
    # "flink depends on the type of phonon scattering in the legs, with 0.5 < flink <= 1 in the absence of 
    # excess noise. For the 300 mK NIST TES bolometers we have measured to date, flink ~ 0.6, but we use 
    # flink = 1 in all the calculations henceforth. Note that for ACT, flink ~ 2 was typical, as in Fig. 1."
tau = 0.5 # default to 0.5

# the inputs from channels.txt are not used in calculating NEP_G, but are relevant in NEP_D, NEP_S, conversion r
fo = open('channels.txt', 'r')
channels = fo.readlines()[4:]
fo.close()

# NEP-NET conversion factors
x = [0.0, 0.0]
r = [0.0, 0.0]
etasky = [0.0, 0.0]
nTES = [0, 0]

nepG2 = [[],[]] # thermal
nepD2 = [[],[]] # dicke
nepS2 = [[],[]] # shot
nep = [[],[]] # total
net_single = [[],[]] # single bolometer
net = [[],[]] # whole array

print('')
print('working file:', label)
print('frequencies: ', end = '')
for i in range(0, len(freq)):
    print(freq[i], end = ' ')
print('\n')

for i in range(0, len(freq)):
    
    if i == 0: # low freq band
        if freq[0] == 90: # MF
            channelline = channels[1] # mf90
            fn = 'opticalparams_mf90.txt'
        elif freq[0] == 150:
            channelline = channels[3] # hf150
            fn = 'opticalparams_hf150.txt'
    elif i == 1: # high freq band
        if freq[1] == 150: # MF
            channelline = channels[2] # mf150
            fn = 'opticalparams_mf150.txt'
        elif freq[1] == 220: # HF
            channelline = channels[4] # hf220
            fn = 'opticalparams_hf220.txt'
            
    channel = channelline.split('\t')
    print('channel:', channel[0])
    print('input:', fn)
    
    freq_bw = float(channel[3]) # frequency bandwidth
    etadet = float(channel[7]) # fraction detectors effective, default to 0.7
    psafety = float(channel[9]) # default to 3.0
    
    # choose the nTES and yieldFrac as one of three conditions
    # 1) parameters from channel.txt info
    # 2) number of detectors functional in the lab tests
    # 3) estimated number of detectors active in the field
    
    #nTES[i] = int(channel[5]) # number of TES in array, according to channels.txt info
    #yieldFrac = float(channel[14]) # default to 0.7
    
    #nTES[i] = ntrials[i] # number of TES in array, according to number of detectors for which there is data
    #yieldFrac = 1 # if the detector produces data, it is functional
    
    # number of TES based on nominal detectors in field, from Steve
    if label[0:3] == 'mf1':
        nTES = [775, 750]
    elif label[0:3] == 'mf2':
        nTES = [690, 640]
    elif label[0:2] == 'hf':
        nTES = [540, 640]
    yieldFrac = 0.7 # supposedly some of these would be broken
    
    fo = open(fn, 'r') 
    etasky[i] = 1.0 # begins at 100% transmission
    lines = fo.readlines()[4:-1]
    for line in lines:
        line = line.split('\t')
        absorb = float(line[6])
        refl = float(line[11])
        etasky[i] *= 1 - absorb - refl
    etasky[i] *= etadet
    fo.close()
    print('etasky = ', etasky[i])

    for j in range(0, ntrials[i]):
        nepG2[i].append(4 * k * Tc[i][j]**2 * G[i][j] * flink)
        nepD2[i].append((Psat[i][j] / psafety)**2 / (freq[i] * 1e9 * freq_bw) / tau) 
        nepS2[i].append(h * freq[i] * 1e9 * (Psat[i][j] / psafety) / tau)
        nep[i].append(np.sqrt(nepG2[i][j] + nepD2[i][j] + nepS2[i][j]))
    
    x[i] = h * freq[i] * 1e9 / k / Tcmb
    r[i] = k * x[i]**2 * np.exp(x[i]) * freq[i] * 1e9 * freq_bw / (np.exp(x[i]) - 1.0)**2
    net_single[i] =  [each / r[i] / np.sqrt(2) / etasky[i] for each in nep[i]]
    net[i] = [each / np.sqrt(yieldFrac * nTES[i]) for each in  net_single[i]]

print('\n')
for i in range(0, len(freq)):
    print('Total NEP,', freq[i], 'GHz: ', np.mean(nep[i]), '\n',
         'NEP from G,', freq[i], 'GHz: ', np.mean([np.sqrt(j) for j in nepG2[i]]), '\n',
         'NEP from Dicke,', freq[i], 'GHz: ', np.mean([np.sqrt(j) for j in nepD2[i]]), '\n',
         'NEP from shot,', freq[i], 'GHz: ', np.mean([np.sqrt(j) for j in nepS2[i]]), '\n',
         'NET single bolometer,', freq[i], 'GHz: ', np.mean(net_single[i]), '\n',
         'NET whole array,', freq[i], 'GHz: ', np.mean(net[i]), '\n')

if debug:
    print('x =', x)
    print('r =', r)
    print('r conversion factor =', [net_single[0][0] / nep[0][0], net_single[1][0] / nep[1][0]])
    print('etasky =', etasky)
    print('nTES =', nTES)
    print('yieldFrac =', yieldFrac)

output.append(
    results(label = label,
            freq = freq,
            Psat = [np.mean(i) for i in Psat],
            G = [np.mean(i) for i in G],
            Tc = [np.mean(i) for i in Tc],
            kappa = [np.mean(i) for i in kappa],
            nep = [np.mean(i) for i in nep],
            nepG = [np.mean([np.sqrt(i) for i in j]) for j in nepG2], 
            nepD = [np.mean([np.sqrt(i) for i in j]) for j in nepD2], 
            nepS = [np.mean([np.sqrt(i) for i in j]) for j in nepS2], 
            net_single = [np.mean(i) for i in net_single],
            net = [np.mean(i) for i in net],
            r = r,
            etasky = etasky,
            nTES = nTES))


working file: hf_steve
frequencies: 150 220 

channel: HFlow
input: opticalparams_hf150.txt
etasky =  0.44902187407963734
channel: HFhigh
input: opticalparams_hf220.txt
etasky =  0.41927948098739487


Total NEP, 150 GHz:  4.97091151753e-17 
 NEP from G, 150 GHz:  2.34288878803e-17 
 NEP from Dicke, 150 GHz:  3.18029147064e-17 
 NEP from shot, 150 GHz:  3.01559464165e-17 
 NET single bolometer, 150 GHz:  0.000236500660344 
 NET whole array, 150 GHz:  1.21642813865e-05 

Total NEP, 220 GHz:  7.22571767637e-17 
 NEP from G, 220 GHz:  3.27914798354e-17 
 NEP from Dicke, 220 GHz:  3.99318997637e-17 
 NEP from shot, 220 GHz:  5.04986988217e-17 
 NET single bolometer, 220 GHz:  0.000282114169947 
 NET whole array, 220 GHz:  1.33286416966e-05 

x = [2.641085770848142, 3.8735924639106085]
r = [3.3099480617406779e-13, 4.31953981607224e-13]
r conversion factor = [4757692015038.1641, 3904306569708.9458]
etasky = [0.44902187407963734, 0.41927948098739487]
nTES = [540, 640]
yieldFrac = 0.7


In [ ]:
# figure generation - for NEP

savefigs = False

binwidth = 7e-19
bins = [np.arange(min(i), max(i) + binwidth, binwidth) for i in nep]

fig = plt.figure()

plt.hist(nep[0], bins = bins[0], alpha = 0.5)
plt.hist(nep[1], bins = bins[1], alpha = 0.5)

if label[0:2] == 'mf':
    netbookouts = open('OutputValues_all_mf90.txt', 'r')
    nep_lowband = float(netbookouts.readlines()[8].split(' ')[1])
    netbookouts.close()
    netbookouts = open('OutputValues_all_mf150.txt', 'r')
    nep_highband = float(netbookouts.readlines()[8].split(' ')[1])
    netbookouts.close()
elif label[0:2] == 'hf':
    netbookouts = open('OutputValues_all_hf150.txt', 'r')
    nep_lowband = float(netbookouts.readlines()[8].split(' ')[1])
    netbookouts.close()
    netbookouts = open('OutputValues_all_hf220.txt', 'r')
    nep_highband = float(netbookouts.readlines()[8].split(' ')[1])
    netbookouts.close()

plt.axvline(x = nep_lowband, color = '#1f77b4', linewidth = 3)
plt.axvline(x = nep_highband, color = '#ff7f0e', linewidth = 3)

plt.title(label)
plt.legend([str(freq[0]) + 'GHz fcast', str(freq[1]) + 'GHz fcast', 
            str(freq[0]) + 'GHz lab', str(freq[1]) + 'GHz lab'])
plt.ylabel('frequency')
plt.xlabel('NEP (W rt(Hz))')
plt.xlim(2e-17, 8.2e-17)
#plt.xlim(2e-17, 1.3e-16) # show 220 GHz forecast for HF, which is large
plt.ticklabel_format(style='sci', axis='x', scilimits=(1,1))
#plt.grid(which = 'major')

plt.text(2.1e-17, 10, str(freq[0]) + 'GHz:' + str(ntrials[0]) + '\n'+ str(freq[1]) + 'GHz:' + str(ntrials[1]))

if savefigs:
    plt.savefig('figures/nep_' + label + '.pdf', bbox_inches = 'tight')

plt.show()

In [ ]:
# figure generation - for NET

savefigs = False

binwidth = 3e-7
bins = [np.arange(min(i), max(i) + binwidth, binwidth) for i in net]

fig = plt.figure()

plt.hist(net[0], bins = bins[0], alpha = 0.5)
plt.hist(net[1], bins = bins[1], alpha = 0.5)

plt.title(label)
plt.legend([str(freq[0]) + 'GHz lab', str(freq[1]) + 'GHz lab'])
plt.ylabel('frequency')
plt.xlabel('NET (K rt(s))')
plt.xlim(0, 2e-5)
plt.ticklabel_format(style='sci', axis='x', scilimits=(1,1))
# plt.grid(which = 'major')

plt.text(2e-6, 5, str(freq[0]) + 'GHz:' + str(ntrials[0]) + '\n'+ str(freq[1]) + 'GHz:' + str(ntrials[1]))

if savefigs:
    plt.savefig('figures/net_' + label + '.pdf', bbox_inches = 'tight')

plt.show()

In [15]:
# write to file comparison of lab data results with forecasts 

output = list(set(output))
print('number of sources processed: ' + str(len(output)))

fo = open('fcastlab_compare.txt', 'w')

wth = '%-12.3e'
wth_text = '%-12.12s'

fo.write('\n' +
         wth_text% 'label' +
         wth_text% 'nu' +
         wth_text% 'Psat' + 
         wth_text% 'G' +
         wth_text% 'Tc' +
         wth_text% 'kappa' +
         wth_text% 'nep' +
         wth_text% 'nepG' +
         wth_text% 'nepD' +
         wth_text% 'nepS' +
         wth_text% 'net_single' +
         wth_text% 'net' + '\n')

fo.write(wth_text% '' +
         wth_text% 'GHz' +
         wth_text% 'W' + 
         wth_text% 'W/K' +
         wth_text% 'K' +
         wth_text% 'UNKNOWN' +
         wth_text% 'W rt(Hz)' +
         wth_text% 'W rt(Hz)' +
         wth_text% 'W rt(Hz)' +
         wth_text% 'W rt(Hz)' +
         wth_text% 'K rt(s)' +
         wth_text% 'K rt(s)' + '\n\n')

fo.write('***LAB DATA***\n\n') 
for item in output:
    for i in range(0, len(freq)):
        fo.write(wth_text% item.label)
        fo.write(wth_text% str(item.freq[i]) +
                 wth% item.Psat[i] + 
                 wth% item.G[i] + 
                 wth% item.Tc[i] +
                 wth% item.kappa[i] + 
                 wth% item.nep[i] + 
                 wth% item.nepG[i] + 
                 wth% item.nepD[i] + 
                 wth% item.nepS[i] + 
                 wth% item.net_single[i] + 
                 wth% item.net[i] + '\n')
    fo.write('\n')

fo.write('***NETBOOK FORECASTS***\n' + 
         '(inputs: each of the five channels in channels.txt, ' +
         'corresponding optical parameters in opticalparams_*.txt files)\n\n')
suffix = ['def', 'mf90', 'mf150', 'hf150', 'hf220']
for item in suffix:
    fn = 'OutputValues_all_' + item + '.txt'
    netbookouts = open(fn, 'r')
    lines = netbookouts.readlines()
    netbookouts.close()
    fo.write(wth_text% lines[1].split(' ')[1].strip() +
             wth_text% lines[2].split(' ')[1].strip() +
             wth% float(lines[4].split(' ')[1]) + 
             wth% float(lines[5].split(' ')[1]) +
             wth% float(lines[6].split(' ')[1]) +
             wth% float(lines[7].split(' ')[1]) +
             wth% float(lines[8].split(' ')[1]) +
             wth% float(lines[9].split(' ')[1]) +
             wth% float(lines[10].split(' ')[1]) +
             wth% float(lines[11].split(' ')[1]) +
             wth% float(lines[13].split(' ')[1]) +
             wth% float(lines[14].split(' ')[1]) + '\n')
fo.write('\n')

fo.write('\n*unknown units for kappa, but kappa was not used in calculation anyway')
fo.write('\n*while netbook forecasts use nTES from channels.txt parameters\n\n')
fo.write('\n' + str(datetime.datetime.now().ctime()) + '\n\n')

fo.close()

number of sources processed: 5


In [16]:
# write to file comparison of safety factor estimate
# method: Psat = fsafety * Ptot (Psat from lab, Ptot from NETbook)

output = list(set(output))
print('number of sources processed: ' + str(len(output)))

fo = open('estimate_safety.txt', 'w')

wth_text = '%-12.12s'

# get all the various ptot in array
suffix = ['mf90', 'mf150', 'hf150', 'hf220']
ptots = []
for item in suffix:
    fn = 'OutputValues_all_' + item + '.txt'
    getptot = open(fn, 'r')
    lines = getptot.readlines()
    ptot = float(lines[16].split(' ')[1])
    ptots.append(ptot)
    getptot.close()
        
fo.write('\nsafety factor estimates, using corresponding channels and optical parameters\n' + 
         'method: Psat = fsafety * Ptot (Psat from lab, Ptot from NETbook)\n' + 
         'default fsafety = 3.0\n\n')
fo.write(wth_text% 'label' + wth_text% 'nu\tpsafety\n\n')
for item in output:
    fo.write(wth_text% item.label)
    if item.freq[0] == 90: # is MF
        fo.write(str(item.freq[0]) + '\t' +
                 str(round(item.Psat[0] / ptots[0], 3)) + '\n' + 
                 wth_text% '' + 
                 str(item.freq[1]) + '\t' +
                 str(round(item.Psat[1] / ptots[1], 3)) + '\n')
    elif item.freq[0] == 150: # is HF
        fo.write(str(item.freq[0]) + '\t' +
                 str(round(item.Psat[0] / ptots[2], 3)) + '\n' +
                 wth_text% '' +
                 str(item.freq[1]) + '\t' + 
                 str(round(item.Psat[1] / ptots[3], 3)) + '\n')
    fo.write('\n')

fo.write('\n' + str(datetime.datetime.now().ctime()) + '\n\n')

fo.close()

number of sources processed: 5


In [17]:
# calculate and write to file comparison the NEP and NET 'infusing NETbook forecast with labtest reality'
# method: NEP^2 = NEP_forecast^2 - NEPG_forecast^2 + NEPG_labtest^2

output = list(set(output))
print('number of sources processed: ' + str(len(output)))

fo = open('fcastlab_synthesise.txt', 'w')

wth = '%-12.3e'
wth_text = '%-12.12s'

suffix = ['mf90', 'mf150', 'hf150', 'hf220']
nep_fcast = []
nepG_fcast = []
for item in suffix:
    fn = 'OutputValues_all_' + item + '.txt'
    netbookouts = open(fn, 'r')
    lines = netbookouts.readlines()
    ptot = float(lines[16].split(' ')[1])
    nep_fcast.append(float(lines[8].split(' ')[1]))
    nepG_fcast.append(float(lines[9].split(' ')[1]))
    netbookouts.close()

fo.write('\nsynthesising NETbook forecast and lab data to produce NEP_tot, NET_single, NET_array ' +
         '\nmethod: NEP^2_synth = NEP_forecast^2 - NEPG_forecast^2 + NEPG_labtest^2\n\n')

fo.write('\n' + 
         wth_text% 'label' +
         wth_text% 'nu' +
         wth_text% 'nep' +
         wth_text% 'net_single' +
         wth_text% 'net' + '\n\n')

fo.write(wth_text% '' +
         wth_text% 'GHz' +
         wth_text% 'W rt(Hz)' +
         wth_text% 'K rt(s)' +
         wth_text% 'K rt(s)' + '\n\n')

for item in output:
    
    if item.freq[0] == 90: # MF
        nep_est = [np.sqrt(nep_fcast[0]**2 - nepG_fcast[0]**2 + item.nepG[0]**2),
                   np.sqrt(nep_fcast[1]**2 - nepG_fcast[1]**2 + item.nepG[1]**2)]
        net_single_est = [nep_est[0] / item.r[0] / np.sqrt(2) / item.etasky[0],
                          (nep_est[1] / item.r[1] / np.sqrt(2) / item.etasky[1])]
        net_est = [net_single_est[0] / np.sqrt(0.7 * nTES[0]), 
                   net_single_est[1] / np.sqrt(0.7 * nTES[1])] # assuming yieldFrac = 0.7
    elif item.freq[0] == 150: # HF      
        nep_est = [np.sqrt(nep_fcast[2]**2 - nepG_fcast[2]**2 + item.nepG[0]**2),
                   np.sqrt(nep_fcast[3]**2 - nepG_fcast[3]**2 + item.nepG[1]**2)]
    fo.write(wth_text% item.label +
             wth_text% str(item.freq[0]) +
             wth% nep_est[0] +
             wth% net_single_est[0] +
             wth% net_est[0] + '\n' +
             wth_text% '' +
             wth_text% str(item.freq[1]) +
             wth% nep_est[1] +
             wth% net_single_est[1] +
             wth% net_est[1] + '\n\n')

fo.write('*as of the creation of this file, the CMB factor, 1/ (r * sqrt(2) * etasky), used in NEP -> ' + 
         'NET_single conversion is calculated within the sensitivity_labtests.ipynb module. It differs ' +
         'from the CMB factor used in NETbook due to small differences in nu (90, 150, 220 here, ' + 
         'channels.txt values in NETbook) and not insignificant differences in etasky (due to NETbook ' + 
         'assuming as the same the absorb and reflect optparams for all LPs (LP40, LP1a, LP1b))\n\n')

fo.write(str(datetime.datetime.now().ctime()) + '\n\n')

fo.close()

number of sources processed: 5
